## Creating a database based on local file with Gen AI

from sqlalchemy import create_engine

# Create an engine instance
engine = create_engine('sqlite:///mydatabase.db')

from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base

# Create the engine
engine = create_engine('sqlite:///mydatabase.db')

# Define a simple model
Base = declarative_base()

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    email = Column(String)

# Create the table
Base.metadata.create_all(engine)

print("Database connected and table created successfully!")

In [1]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, DateTime, Float
from sqlalchemy.orm import sessionmaker, declarative_base

In [2]:
Base = declarative_base()

In [3]:
class User(Base):
    __tablename__ = 'users'

    user_id = Column(Integer, primary_key=True)
    username = Column(String)
    email = Column(String, unique=True)
    password_hash = Column(String)
    first_name = Column(String)
    last_name = Column(String)
    address = Column(String)
    phone_number = Column(String)

class Product(Base):
    __tablename__ = 'products'

    product_id = Column(Integer, primary_key=True)
    name = Column(String)
    description = Column(String)
    price = Column(Float)
    quantity_in_stock = Column(Integer)
    category_id = Column(Integer, ForeignKey('categories.category_id'))
    image_url = Column(String)

class Category(Base):
    __tablename__ = 'categories'

    category_id = Column(Integer, primary_key=True)
    name = Column(String)

class Order(Base):
    __tablename__ = 'orders'

    order_id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.user_id'))
    order_date = Column(DateTime)
    total_amount = Column(Float)
    shipping_address = Column(String)
    billing_address = Column(String)

class OrderItem(Base):
    __tablename__ = 'order_items'

    order_item_id = Column(Integer, primary_key=True)
    order_id = Column(Integer, ForeignKey('orders.order_id'))
    product_id = Column(Integer, ForeignKey('products.product_id'))
    quantity = Column(Integer)
    price = Column(Float)

In [4]:
engine = create_engine('sqlite:///ecommerce.db')

In [5]:
Base.metadata.create_all(engine)

In [23]:
Session = sessionmaker(bind=engine)
session = Session()

In [12]:
# Insert a new user
new_user = User(username='jane_doe', email='janedoe@example.com', password_hash='hashed_password')
session.add(new_user)

# Commit changes
session.commit()


In [24]:

# Query for all users
users = session.query(User).all()

for user in users:
    for column_name in User.__table__.columns.keys():
        print(getattr(user, column_name))

# Convert to Pandas DataFrame
import pandas as pd

df = pd.DataFrame(map(lambda x: x.__dict__, users))

df

2
jane_doe
janedoe@example.com
hashed_password
None
None
None
None


,_sa_instance_state,first_name,password_hash,email,address,username,user_id,last_name,phone_number
0,<sqlalchemy.orm.state.InstanceState object at ...,None,hashed_password,janedoe@example.com,None,jane_doe,2,None,None


In [10]:
users = session.query(User) \
    .order_by(User.user_id) \
    .limit(10) \
    .all()

In [16]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

def update_record(engine, model, update_dict, filter_dict):
  """Updates a record in the database based on a filter.

  Args:
    engine: The SQLAlchemy engine.
    model: The SQLAlchemy model class.
    update_dict: A dictionary containing the fields to update and their new values.
    filter_dict: A dictionary containing the filter criteria to identify the record to update.

  Returns:
    None
  """

  Session = sessionmaker(bind=engine)
  session = Session()

  # Query for the record to update
  query = session.query(model).filter_by(**filter_dict)
  record = query.first()

  # Update the record's attributes
  for key, value in update_dict.items():
    setattr(record, key, value)

  # Commit the changes
  session.commit()
  session.close()

In [15]:
# Assuming you have a User model with id, name, and email fields
user_id = 1  # The ID of the user to update
update_data = {'username': 'Zorro', 'email': 'zorro@example.com'}

update_record(engine, User, update_data, {'user_id': user_id})

In [21]:
def delete_user_by_id(user_id):
    Session = sessionmaker(bind=engine)
    session = Session()

    user_to_delete = session.query(User).filter_by(user_id=user_id).first()

    if user_to_delete:
        session.delete(user_to_delete)
        session.commit()
        print(f"User with ID {user_id} deleted successfully.")
    else:
        print(f"User with ID {user_id} not found.")

    session.close()

In [22]:
# Example usage:
user_id_to_delete = 1
delete_user_by_id(user_id_to_delete)

User with ID 1 deleted successfully.


In [ ]:

# Query for products in a specific category
products_in_category = session.query(Product).all()

In [18]:
session.close()

In [22]:
from sqlalchemy import MetaData, create_engine

engine = create_engine('sqlite:///ecommerce.db')
metadata = MetaData()
metadata.reflect(bind=engine)

# Access tables and columns:
for table in metadata.tables.values():
    print(table.name)
    for column in table.columns:
        print(f"  - {column.name}: {column.type}")

categories
  - category_id: INTEGER
  - name: VARCHAR
order_items
  - order_item_id: INTEGER
  - order_id: INTEGER
  - product_id: INTEGER
  - quantity: INTEGER
  - price: FLOAT
orders
  - order_id: INTEGER
  - user_id: INTEGER
  - order_date: DATETIME
  - total_amount: FLOAT
  - shipping_address: VARCHAR
  - billing_address: VARCHAR
users
  - user_id: INTEGER
  - username: VARCHAR
  - email: VARCHAR
  - password_hash: VARCHAR
  - first_name: VARCHAR
  - last_name: VARCHAR
  - address: VARCHAR
  - phone_number: VARCHAR
products
  - product_id: INTEGER
  - name: VARCHAR
  - description: VARCHAR
  - price: FLOAT
  - quantity_in_stock: INTEGER
  - category_id: INTEGER
  - image_url: VARCHAR
